# Install deps

In [ ]:
%pip install -q ultralytics torch torchvision opencv-python-headless scipy plyfile pyyaml tqdm matplotlib pandas scikit-learn trimesh

import torch, yaml, numpy as np, cv2
from pathlib import Path
import trimesh

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 740.4/740.4 kB 25.5 MB/s eta 0:00:00


# Setup directories

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

ZIP_ROOT = Path('/content/drive/My Drive/project_6')         # <- put your directory here
DRIVE_ROOT = Path('/content/drive/My Drive/pinhole_project') # <- put your directory here
LM_ZIP = ZIP_ROOT / 'Linemod_preprocessed.zip'
LM_ROOT = Path('/content/Linemod_preprocessed')
LM_DATA = LM_ROOT / 'data'
MODELS_DIR = LM_ROOT / 'models'
YOLO_RUN = DRIVE_ROOT / 'yolo' / 'linemod'
YOLO_BEST = YOLO_RUN / 'weights' / 'best.pt'
POSE_CKPT_DIR = DRIVE_ROOT / 'checkpoints'
POSE_BEST = POSE_CKPT_DIR / 'best_pose.pt'
POSE_LAST = POSE_CKPT_DIR / 'last_pose.pt'
EVAL_DIR = DRIVE_ROOT / 'eval_results'
VIZ_DIR = DRIVE_ROOT / 'visualizations'
for p in [POSE_CKPT_DIR, EVAL_DIR, VIZ_DIR, YOLO_RUN]:
    p.mkdir(parents=True, exist_ok=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from importlib.machinery import SourceFileLoader
import sys
import os
import json
import zipfile
import shutil

sys.path.append(str(DRIVE_ROOT))
prep_mod = SourceFileLoader('prepare_data', str(
    DRIVE_ROOT / 'code' / 'prepare_data.py')).load_module()
train_yolo_mod = SourceFileLoader('train_yolo_linemod', str(
    DRIVE_ROOT / 'code' / 'train_yolo_linemod.py')).load_module()
infer_yolo_mod = SourceFileLoader('run_yolo_inference', str(
    DRIVE_ROOT / 'code' / 'run_yolo_inference.py')).load_module()
dataset_mod = SourceFileLoader('dataset', str(
    DRIVE_ROOT / 'code' / 'dataset.py')).load_module()
pose_model_mod = SourceFileLoader('pose_net_rgb_geometric', str(
    DRIVE_ROOT / 'code' / 'pose_net_rgb_geometric.py')).load_module()
train_pose_mod = SourceFileLoader('train_posenet_rgb', str(
    DRIVE_ROOT / 'code' / 'train_posenet_rgb.py')).load_module()
add_mod = SourceFileLoader('add_posenet', str(
    DRIVE_ROOT / 'code' / 'add_posenet.py')).load_module()


prepare_dataset = prep_mod.prepare_dataset
fine_tune_yolo = train_yolo_mod.fine_tune_yolo
run_yolo_inference = infer_yolo_mod.run_yolo_inference
train_pose_net = train_pose_mod.train_pose_net
run_add_eval = add_mod.run_add_eval
LineModDataset = dataset_mod.LineModDataset
PoseNetRGBGeometric = pose_model_mod.PoseNetRGBGeometric

# Extract dataset

In [ ]:
if LM_ZIP.exists() and not LM_ROOT.exists():
    LM_ROOT.mkdir(parents=True, exist_ok=True)
    print('Extracting Linemod [3-4 minutes ...]')
    with zipfile.ZipFile(LM_ZIP, 'r') as z:
        z.extractall('/content')

Extracting Linemod [3-4 minutes ...]


# Run prepare_dataset to build YOLO splits + linemod_multi.yaml

In [ ]:
base_data = Path('/content/data')
base_data.mkdir(parents=True, exist_ok=True)
linemod_symlink = base_data / 'linemod'
if linemod_symlink.exists() or linemod_symlink.is_symlink():
    try:
        linemod_symlink.unlink()
    except Exception:
        shutil.rmtree(linemod_symlink, ignore_errors=True)
linemod_symlink.symlink_to(LM_DATA, target_is_directory=True)

os.chdir(base_data)
prepare_dataset()
os.chdir('/content')
prep_summary = {
    'yaml_path': str(base_data / 'linemod_multi.yaml'),
    'yolo_prep_dir': str(base_data / 'linemod_yolo')
}
with open('/content/prep_summary.json', 'w') as f:
    json.dump(prep_summary, f, indent=2)
print('Prep done:', prep_summary)

# Train YOLO

In [ ]:
data_yaml = Path('/content/data/linemod_multi.yaml')
device = 'cuda' if torch.cuda.is_available() else 'cpu'

if YOLO_BEST.exists():
    print('Using existing YOLO weights:', YOLO_BEST)
else:
    _model, _results = fine_tune_yolo(
        pretrained_weights='yolov8n.pt',
        data_yaml=str(data_yaml),
        device=device,
        epochs=20,
        batch=32,
        imgsz=640,
        freeze=10,
        workers=2,
        project=str(YOLO_RUN.parent),
        name=YOLO_RUN.name
    )
    YOLO_BEST.parent.mkdir(parents=True, exist_ok=True)
print('YOLO best path:', YOLO_BEST)

# YOLO inference

In [ ]:
import yaml as _yaml
dataset_root_for_func = str(LM_ROOT)
full_pred_path = LM_DATA / 'yolo_predictions_full.yaml'
simple_pred_path = LM_DATA / 'yolo_predicted_boxes.yaml'

preds = run_yolo_inference(
    yolo_weights_path=str(YOLO_BEST),
    dataset_root=dataset_root_for_func,
    output_path=str(full_pred_path),
    conf_threshold=0.25,
    device=device,
)

with open(full_pred_path, 'r') as f:
    rich_preds = _yaml.safe_load(f) or {}
simple_preds = {}
for k, v in rich_preds.items():
    try:
        folder, sample = k.split('/')
        key = f"{folder}/rgb/{int(sample):04d}.png"
        simple_preds[key] = v['bbox']
    except Exception:
        continue
with open(simple_pred_path, 'w') as f:
    _yaml.safe_dump(simple_preds, f)
print('Saved simple boxes to', simple_pred_path)

# PoseNet train preparation

In [ ]:
pred_boxes_drive = DRIVE_ROOT / 'yolo_predicted_boxes.yaml'
pred_boxes_full = DRIVE_ROOT / 'yolo_predictions_full.yaml'
# pred_boxes_path = str(simple_pred_path) # if you do the YOLO train in one session

pred_boxes_path = str(pred_boxes_drive)   # if you already have the .yaml
val_split = 0.2
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Pred boxes:', pred_boxes_path)
print('Val split:', val_split)
print('Device:', device)

## PoseNet train RUN

In [ ]:
pose_model, pose_history = train_pose_net(
    root_dir=str(LM_DATA),
    models_dir=str(MODELS_DIR),
    pred_boxes_path=pred_boxes_path,
    epochs=110,
    batch_size=32,
    learning_rate=1e-4,
    weight_decay=1e-6,
    lambda_rot=1.0,
    lambda_trans=1.0,
    val_split=val_split,
    add_thresh=0.02,
    num_workers=2,
    seed=42,
    device=device,
    checkpoint_best=str(POSE_BEST),  # where to save best model
    checkpoint_last=str(POSE_LAST),  # where to save last model
    resume_from=str(POSE_LAST),      # None or str to resume from
    eval_only=False,                 # set to True to only evaluate
    scheduler_patience=5,            # for ReduceLROnPlateau (0 to disable)
    early_stop_patience=0,           # early stopping (0 to disable)
    optimizer_type='adamw',          # optimizer type: 'adam' or 'adamw'
    freeze_backbone_ratio_val=0.8,   # fraction of backbone layers to freeze
    unfreeze_epoch=100,              # epoch to unfreeze backbone
    rotation_loss_type='geodesic',
)

# PoseNet evaluation & Best pose predictions.yaml

In [ ]:
import pandas as pd
per_class, overall = run_add_eval(
    root_dir=str(LM_DATA),
    models_dir=str(MODELS_DIR),
    checkpoint=str(POSE_BEST),
    pred_boxes_path=pred_boxes_path,
    batch_size=32,
    val_split=val_split,
    num_workers=2,
    seed=42,
    device=device,
    output_csv=str(EVAL_DIR / 'add_results.csv'),
    save_predictions=str(EVAL_DIR / 'best_pose_predictions.yaml'),
)

per_class_df = pd.DataFrame.from_dict(per_class, orient='index')
per_class_csv = EVAL_DIR / 'add_by_class.csv'
per_class_df.to_csv(per_class_csv)
metrics_json = EVAL_DIR / 'metrics.json'
with open(metrics_json, 'w') as f:
    json.dump({'overall': overall, 'per_class': per_class}, f, indent=2)
print('Saved:', per_class_csv, metrics_json)

# PoseNet ADD-S evaluation (Symmetric)

In [ ]:
adds_mod = SourceFileLoader('adds_posenet', str(
    DRIVE_ROOT / 'code' / 'adds_posenet.py')).load_module()
run_adds_eval = adds_mod.run_adds_eval

per_class_adds, overall_adds = run_adds_eval(
    root_dir=str(LM_DATA),
    models_dir=str(MODELS_DIR),
    checkpoint=str(POSE_BEST),
    pred_boxes_path=pred_boxes_path,
    batch_size=32,
    val_split=val_split,
    num_workers=2,
    seed=42,
    device=device,
    output_csv=str(EVAL_DIR / 'adds_results_2classes.csv'),
    symmetric_only=True,
)

per_class_adds_df = pd.DataFrame.from_dict(per_class_adds, orient='index')
per_class_adds_csv = EVAL_DIR / 'adds_by_class.csv'
per_class_adds_df.to_csv(per_class_adds_csv)

metrics_adds_json = EVAL_DIR / 'metrics_adds.json'
with open(metrics_adds_json, 'w') as f:
    json.dump({'overall': overall_adds, 'per_class': per_class_adds}, f, indent=2)

print(f'\n✓ ADD-S evaluation completed!')
print(f'  - CSV summary: {EVAL_DIR / "adds_results.csv"}')
print(f'  - Per-class CSV: {per_class_adds_csv}')
print(f'  - Metrics JSON: {metrics_adds_json}')

# Visualize samples with bounding boxes and 3D axes

In [ ]:
viz_mod = SourceFileLoader('visualize_samples', str(
    DRIVE_ROOT / 'code' / 'visualize_samples.py')).load_module()
visualize_random_samples = viz_mod.visualize_random_samples

pose_pred_file = EVAL_DIR / 'best_pose_predictions.yaml'

visualize_random_samples(
    root_dir=str(LM_DATA),
    output_dir=str(VIZ_DIR),
    axis_length=0.2,
    yolo_pred_path=str(
        pred_boxes_full) if pred_boxes_full.exists() else None,
    pose_pred_path=str(pose_pred_file) if pose_pred_file.exists() else None,
)
print(f'Visualizations saved to {VIZ_DIR}')